## 1. Imports and Problem Generator

In [1]:
from itertools import product
import numpy as np
import time
import csv
import json

In [2]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """
    Generate a weighted directed graph as a distance matrix.
    
    Parameters:
    - size: Number of nodes in the graph
    - density: Probability of edge existence (0.0 to 1.0)
    - negative_values: Allow negative edge weights
    - noise_level: Random noise added to distances
    - seed: Random seed for reproducibility
    
    Returns:
    - Distance matrix where matrix[i][j] is the weight from node i to j
      (np.inf indicates no edge exists)
    """
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    
    if negative_values:
        problem = problem * 2 - 1
    
    problem *= noise_level
    
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

## 2. Algorithm Implementations

In [3]:
def dijkstra(problem, start):
    """
    Standard Dijkstra's algorithm - O(V²) complexity.
    WARNING: May produce incorrect results with negative edge weights.
    
    Parameters:
    - problem: Distance matrix (np.ndarray)
    - start: Starting node index
    
    Returns:
    - distances: Dict mapping node -> shortest distance from start
    - paths: Dict mapping node -> shortest path from start
    """
    n = problem.shape[0]
    
    distances = {i: np.inf for i in range(n)}
    paths = {i: [] for i in range(n)}
    distances[start] = 0
    paths[start] = [start]
    
    unvisited = set(range(n))
    
    while unvisited:
        current = min(unvisited, key=lambda node: distances[node])
        
        if distances[current] == np.inf:
            break
        
        unvisited.remove(current)
        
        for neighbor in range(n):
            if neighbor in unvisited and problem[current][neighbor] != np.inf:
                new_distance = distances[current] + problem[current][neighbor]
                
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    paths[neighbor] = paths[current] + [neighbor]
    
    return distances, paths

In [4]:
def dijkstra_positive_paths(problem, start):
    """
    Modified Dijkstra that only accepts paths with non-negative total distance.
    Safe for graphs with negative edges - ensures all returned distances are >= 0.
    
    Parameters:
    - problem: Distance matrix (np.ndarray)
    - start: Starting node index
    
    Returns:
    - distances: Dict mapping node -> shortest NON-NEGATIVE distance
    - paths: Dict mapping node -> path with non-negative total distance
    """
    n = problem.shape[0]
    
    distances = {i: np.inf for i in range(n)}
    paths = {i: [] for i in range(n)}
    distances[start] = 0
    paths[start] = [start]
    
    unvisited = set(range(n))
    
    while unvisited:
        current = min(unvisited, key=lambda node: distances[node])
        
        if distances[current] == np.inf:
            break
        
        unvisited.remove(current)
        
        for neighbor in range(n):
            if neighbor in unvisited and problem[current][neighbor] != np.inf:
                new_distance = distances[current] + problem[current][neighbor]
                
                # Only accept non-negative paths
                if new_distance >= 0 and new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    paths[neighbor] = paths[current] + [neighbor]
    
    return distances, paths

In [5]:
def bellman_ford(problem, start):
    """
    Bellman-Ford algorithm - O(V³) complexity.
    Handles negative edges and detects negative cycles.
    
    Parameters:
    - problem: Distance matrix (np.ndarray)
    - start: Starting node index
    
    Returns:
    - distances: Dict mapping node -> shortest distance
    - paths: Dict mapping node -> shortest path
    - has_negative_cycle: Boolean indicating cycle detection
    - negative_cycle_nodes: Set of nodes affected by negative cycles
    """
    n = problem.shape[0]
    
    distances = {i: np.inf for i in range(n)}
    paths = {i: [] for i in range(n)}
    distances[start] = 0
    paths[start] = [start]
    
    # Relax edges |V| - 1 times
    for iteration in range(n - 1):
        updated = False
        for u in range(n):
            if distances[u] == np.inf:
                continue
            for v in range(n):
                if problem[u][v] != np.inf:
                    new_distance = distances[u] + problem[u][v]
                    if new_distance < distances[v]:
                        distances[v] = new_distance
                        paths[v] = paths[u] + [v]
                        updated = True
        
        if not updated:
            break
    
    # Check for negative cycles (Vth iteration)
    has_negative_cycle = False
    negative_cycle_nodes = set()
    
    for u in range(n):
        if distances[u] == np.inf:
            continue
        for v in range(n):
            if problem[u][v] != np.inf:
                if distances[u] + problem[u][v] < distances[v]:
                    has_negative_cycle = True
                    negative_cycle_nodes.add(v)
    
    # Propagate negative cycle influence
    if has_negative_cycle:
        for _ in range(n):
            new_affected = set()
            for u in negative_cycle_nodes:
                for v in range(n):
                    if problem[u][v] != np.inf and v not in negative_cycle_nodes:
                        new_affected.add(v)
            negative_cycle_nodes.update(new_affected)
            if not new_affected:
                break
    
    return distances, paths, has_negative_cycle, negative_cycle_nodes

## 3. Problem Generation

Generate 224 test problems with varying parameters:

In [7]:
# Problem parameters
sizes = [10, 20, 50, 100, 200, 500, 1000]
densities = [0.2, 0.5, 0.8, 1.0]
noise_levels = [0, 0.1, 0.5, 0.8]
negative_values_options = [False, True]

problems = []

for size in sizes:
    for density in densities:
        for noise in noise_levels:
            for neg_val in negative_values_options:
                problem_config = {
                    'size': size,
                    'density': density,
                    'noise_level': noise,
                    'negative_values': neg_val,
                    'problem': create_problem(
                        size=size,
                        density=density,
                        noise_level=noise,
                        negative_values=neg_val,
                        seed=42
                    )
                }
                problems.append(problem_config)

print(f"Generated {len(problems)} problems")
print(f"\nParameter combinations:")
print(f"  Sizes: {sizes}")
print(f"  Densities: {densities}")
print(f"  Noise levels: {noise_levels}")
print(f"  Negative values: {negative_values_options}")
print(f"\nTotal: {len(sizes)} × {len(densities)} × {len(noise_levels)} × {len(negative_values_options)} = {len(problems)}")

Generated 224 problems

Parameter combinations:
  Sizes: [10, 20, 50, 100, 200, 500, 1000]
  Densities: [0.2, 0.5, 0.8, 1.0]
  Noise levels: [0, 0.1, 0.5, 0.8]
  Negative values: [False, True]

Total: 7 × 4 × 4 × 2 = 224


## 4. Solve All Problems with Dijkstra positive-path-only

Solve all 224 problems and collect performance metrics:

In [ ]:
results = []

print("Solving all problems with Dijkstra positive-path-only...\n")

for i, prob in enumerate(problems):
    if (i + 1) % 50 == 0:
        print(f"Progress: {i+1}/{len(problems)} problems solved")
    
    # Time the execution
    start_time = time.time()
    distances, paths = dijkstra_positive_paths(prob['problem'], start=0)
    execution_time = time.time() - start_time
    
    # Calculate statistics
    reachable_distances = [d for d in distances.values() if d != np.inf and d != 0]
    
    result = {
        'size': prob['size'],
        'density': prob['density'],
        'noise_level': prob['noise_level'],
        'negative_values': prob['negative_values'],
        'execution_time': execution_time,
        'reachable_nodes': len(reachable_distances),
        'unreachable_nodes': sum(1 for d in distances.values() if d == np.inf),
        'avg_distance': np.mean(reachable_distances) if reachable_distances else 0,
        'max_distance': max(reachable_distances) if reachable_distances else 0,
        'min_distance': min(reachable_distances) if reachable_distances else 0
    }
    
    results.append(result)

print(f"\n✓ Solved {len(results)} problems successfully!")
print(f"\nPerformance Summary:")
print(f"  Total execution time: {sum(r['execution_time'] for r in results):.2f} seconds")
print(f"  Average execution time: {np.mean([r['execution_time'] for r in results]):.4f} seconds")
print(f"  Fastest solve: {min(results, key=lambda r: r['execution_time'])['execution_time']:.6f} seconds")
print(f"  Slowest solve: {max(results, key=lambda r: r['execution_time'])['execution_time']:.4f} seconds")

Solving all problems with Standard Dijkstra...

Progress: 50/224 problems solved
Progress: 100/224 problems solved
Progress: 150/224 problems solved
Progress: 150/224 problems solved
Progress: 200/224 problems solved
Progress: 200/224 problems solved

✓ Solved 224 problems successfully!

Performance Summary:
  Total execution time: 12.22 seconds
  Average execution time: 0.0546 seconds
  Fastest solve: 0.000030 seconds
  Slowest solve: 1.2108 seconds

✓ Solved 224 problems successfully!

Performance Summary:
  Total execution time: 12.22 seconds
  Average execution time: 0.0546 seconds
  Fastest solve: 0.000030 seconds
  Slowest solve: 1.2108 seconds


## 5. Export Results to CSV and JSON

In [14]:
# Export to CSV
csv_filename = 'dijkstra_results.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['size', 'density', 'noise_level', 'negative_values', 'execution_time', 
                  'reachable_nodes', 'unreachable_nodes', 'avg_distance', 'max_distance', 'min_distance']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"✓ Results saved to {csv_filename}")

# Export to JSON
json_filename = 'dijkstra_results.json'
with open(json_filename, 'w') as jsonfile:
    json.dump(results, jsonfile, indent=2)

print(f"✓ Results saved to {json_filename}")

✓ Results saved to dijkstra_results.csv
✓ Results saved to dijkstra_results.json


## 6. Interactive Problem Explorer

Analyze individual problems with all three algorithms and compare results:

In [17]:
# ============================================================================
# INTERACTIVE PROBLEM ANALYZER
# Change problem_index to analyze different problems (0 to 223)
# ============================================================================

problem_index = 95  # <-- Change this to select a problem

print("="*80)
print(f"PROBLEM {problem_index + 1} - COMPREHENSIVE ANALYSIS")
print("="*80)

prob = problems[problem_index]

print(f"\nConfiguration:")
print(f"  Size: {prob['size']} nodes")
print(f"  Density: {prob['density']}")
print(f"  Noise level: {prob['noise_level']}")
print(f"  Negative values: {prob['negative_values']}")

# Run all three algorithms
print(f"\n" + "-"*80)
print("RUNNING ALGORITHMS")
print("-"*80)

print(f"\n[1/3] Standard Dijkstra...")
start_time = time.time()
distances, paths = dijkstra(prob['problem'], start=0)
dijkstra_time = time.time() - start_time
print(f"      Execution time: {dijkstra_time:.6f} seconds")

print(f"\n[2/3] Positive-Path-Only Dijkstra...")
start_time = time.time()
pos_distances, pos_paths = dijkstra_positive_paths(prob['problem'], start=0)
pos_dijkstra_time = time.time() - start_time
print(f"      Execution time: {pos_dijkstra_time:.6f} seconds")

print(f"\n[3/3] Bellman-Ford...")
start_time = time.time()
bf_distances, bf_paths, has_neg_cycle, neg_cycle_nodes = bellman_ford(prob['problem'], start=0)
bf_time = time.time() - start_time
print(f"      Execution time: {bf_time:.6f} seconds")

# Graph statistics
matrix = prob['problem']
finite_edges = matrix[~np.isinf(matrix) & (matrix != 0)]

print(f"\n" + "="*80)
print("GRAPH STATISTICS")
print("="*80)
print(f"Total nodes: {prob['size']}")
print(f"Finite edges: {len(finite_edges)}")
if len(finite_edges) > 0:
    print(f"Edge weight range: [{finite_edges.min():.2f}, {finite_edges.max():.2f}]")
    print(f"Average edge weight: {finite_edges.mean():.2f}")
    if finite_edges.min() < 0:
        print(f"⚠️  Contains negative edges (min: {finite_edges.min():.2f})")

# Reachability
reachable_distances = [d for d in distances.values() if d != np.inf and d != 0]
unreachable = [node for node, d in distances.items() if d == np.inf]

print(f"\nReachability from node 0:")
print(f"  Reachable: {len(reachable_distances)}/{prob['size']-1} nodes")
print(f"  Unreachable: {len(unreachable)} nodes")

print(f"\n" + "="*80)
print("ALGORITHM COMPARISON")
print("="*80)
print(f"\n{'Node':<8} {'Standard':<14} {'Positive-Only':<14} {'Bellman-Ford':<14} {'Status'}")
print("-" * 80)

for node in range(min(prob['size'], 20)):  # Show first 20 nodes
    if node == 0:
        continue
    
    std_dist = distances[node]
    pos_dist = pos_distances[node]
    bf_dist = bf_distances[node]
    
    # Determine status
    if node in neg_cycle_nodes:
        status = "⚠️  Negative cycle"
    elif np.isinf(pos_dist):
        status = "No positive path"
    elif pos_dist != std_dist:
        status = "Negative path exists"
    else:
        status = "✓ OK"
    
    # Format distances
    std_str = f"{std_dist:.2f}" if std_dist != np.inf else "∞"
    pos_str = f"{pos_dist:.2f}" if pos_dist != np.inf else "∞"
    bf_str = f"{bf_dist:.2f}" if bf_dist != np.inf else "∞"
    if node in neg_cycle_nodes:
        bf_str = "-∞"
    
    print(f"{node:<8} {std_str:<14} {pos_str:<14} {bf_str:<14} {status}")

if prob['size'] > 20:
    print(f"\n(Showing first 20 nodes only. Total nodes: {prob['size']})")

print(f"\n" + "="*80)
print("PERFORMANCE SUMMARY")
print("="*80)
print(f"Algorithm execution times:")
print(f"  Standard Dijkstra:      {dijkstra_time:.6f} seconds")
print(f"  Positive-Only Dijkstra: {pos_dijkstra_time:.6f} seconds")
print(f"  Bellman-Ford:           {bf_time:.6f} seconds ({bf_time/dijkstra_time:.1f}x slower)")

print(f"\n" + "="*80)
print("SAFETY ASSESSMENT")
print("="*80)

if has_neg_cycle:
    print(f"⚠️  WARNING: Negative cycles detected!")
    print(f"   Affected nodes: {sorted(neg_cycle_nodes)}")
    print(f"\n✓  RECOMMENDATION: Use Positive-Path-Only Dijkstra for safe results")
else:
    print(f"✓ No negative cycles detected")
    print(f"✓ All three algorithms produce identical results")
    print(f"✓ Standard Dijkstra is optimal for this problem")

print(f"\n" + "="*80)
print(f"To analyze a different problem, change 'problem_index' (valid range: 0-{len(problems)-1})")
print("="*80)

PROBLEM 96 - COMPREHENSIVE ANALYSIS

Configuration:
  Size: 50 nodes
  Density: 1.0
  Noise level: 0.8
  Negative values: True

--------------------------------------------------------------------------------
RUNNING ALGORITHMS
--------------------------------------------------------------------------------

[1/3] Standard Dijkstra...
      Execution time: 0.002447 seconds

[2/3] Positive-Path-Only Dijkstra...
      Execution time: 0.001476 seconds

[3/3] Bellman-Ford...
      Execution time: 0.111603 seconds

GRAPH STATISTICS
Total nodes: 50
Finite edges: 2449
Edge weight range: [-728.00, 1833.00]
Average edge weight: 497.80
⚠️  Contains negative edges (min: -728.00)

Reachability from node 0:
  Reachable: 49/49 nodes
  Unreachable: 0 nodes

ALGORITHM COMPARISON

Node     Standard       Positive-Only  Bellman-Ford   Status
--------------------------------------------------------------------------------
1        -13539.00      26.00          -∞             ⚠️  Negative cycle
2        -